## Compulsory Task 

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [62]:
# Load up store_income_data.csv

import pandas as pd
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import process
import chardet
import datetime
from datetime import date

df = pd.read_csv('store_income_data_task.csv')

1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [63]:
df.head()

df['country'] = df['country'].str.lower()

df['country'] = df['country'].str.strip()
df['country'] = df['country'].str.strip('/')
df['country'] = df['country'].str.strip('.')

countries = df['country'].unique()
print(f"There are {len(countries)} unique countries")
countries


There are 13 unique countries


array(['united states', 'britain', 'united kingdom', 'u.k', 'sa',
       'america', nan, 's.a', 'england', 'uk', '',
       'united states of america', 's. africasouth africa'], dtype=object)

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [64]:
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# Step 3: Replace country variations with consistent names

# Define a function to replace matches in the "country" column based on a minimum similarity ratio
def replace_matches_in_column(df, column, string_to_match, min_ratio=90):
    # Get unique country names
    unique_strings = df[column].unique()
    
    # Use fuzzy matching to find similar country names
    matches = process.extract(string_to_match, unique_strings, limit=100, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    
    # Filter matches that meet the minimum ratio
    close_matches = [match[0] for match in matches if match[1] >= min_ratio]
    
    # Replace close matches with the specified standard country name
    df.loc[df[column].isin(close_matches), column] = string_to_match

df['country'].replace(['', 'nan', None], np.nan, inplace=True)
                      
# Apply the replacement function to ensure only three standard country names remain
replace_matches_in_column(df, 'country', 'south africa')
replace_matches_in_column(df, 'country', 'united kingdom')
replace_matches_in_column(df, 'country', 'united states')

df.replace('britain', 'united kingdom', inplace=True)
df.replace('u.k', 'united kingdom', inplace=True)
df.replace('uk', 'united kingdom', inplace=True)
df.replace('england', 'united kingdom', inplace=True)
df.replace('sa', 'south africa', inplace=True)
df.replace('s.a', 'south africa', inplace=True)
df.replace('s. africasouth africa', 'south africa', inplace=True)
df.replace('america', 'united states', inplace=True)
df.replace('united states of america', 'united states', inplace=True)

df.dropna(subset=['country'], inplace=True)

unique_countries = df['country'].unique()
print(f"Final unique country values: {unique_countries}")


Final unique country values: ['united states' 'united kingdom' 'south africa']


/var/folders/m7/3y4fdpjn45qdddmm1wfk91x40000gn/T/ipykernel_60953/2709251610.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['country'].replace(['', 'nan', None], np.nan, inplace=True)


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [65]:

# Convert 'date_measured' to datetime format
df['date_measured'] = pd.to_datetime(df['date_measured'], format='%d-%m-%Y')


# Calculate the difference in days from the current date
current_date = pd.to_datetime(datetime.date.today())
df['days_ago'] = (current_date - df['date_measured']).dt.days

# Display the first few rows to verify the 'days_ago' calculation
df.head()


,id,store_name,store_email,department,income,date_measured,country,days_ago
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,2006-02-04,united states,6862
1,2,Nordson Corporation,NaN,Tools,$41744177.01,2006-01-04,united kingdom,6893
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,2003-09-12,united states,7738
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,2006-05-08,united kingdom,6769
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,1973-01-21,united kingdom,18929
